# Solar Photovoltaic Module

Models on-site PV electricity generation with temperature-dependent efficiency and exergy analysis using the Petela-Landsberg-Press relation.

## Energy Analysis

**Solar energy input:**
$$\dot{E}_{solar,in} = G \cdot A_{PV}$$

**Cell temperature (NOCT model):**
$$T_{cell} = T_{amb} + \left(\frac{NOCT - 20}{800}\right) G$$

**Temperature-dependent efficiency:**
$$\eta_{elec}(T_{cell}) = \eta_{ref} \left[1 - \beta_{ref}(T_{cell} - T_{ref})\right]$$

**Electrical power output:**
$$\dot{W}_{elec} = \eta_{elec}(T_{cell}) \cdot \dot{E}_{solar,in}$$

Typical $\eta_{ref} = 0.15 - 0.22$, $\beta_{ref} \approx 0.004 \ K^{-1}$

## Exergy Analysis

**Solar exergy factor (Petela-Landsberg-Press):**
$$\psi_{solar} = 1 - \frac{4}{3}\left(\frac{T_0}{T_{sun}}\right) + \frac{1}{3}\left(\frac{T_0}{T_{sun}}\right)^4$$

With $T_{sun} = 5800\ K$, $T_0 = 298\ K$: $\psi_{solar} \approx 0.93$

**Solar exergy input:**
$$\dot{Ex}_{solar,in} = \psi_{solar} \cdot G \cdot A_{PV}$$

**Electrical exergy output (pure exergy):**
$$\dot{Ex}_{elec} = \dot{W}_{elec}$$

**Exergy efficiency:**
$$\Psi_{PV} = \frac{\dot{W}_{elec}}{\psi_{solar} \cdot G \cdot A_{PV}}$$

**Exergy destruction:**
$$\dot{Ex}_{d,PV} = \dot{Ex}_{solar,in} - \dot{Ex}_{elec}$$

## Parameters

In [1]:
# Solar PV Default Parameters
const PV_DEFAULTS = (
    η_ref = 0.20,           # Reference efficiency at STC [-]
    T_ref = 298.15,         # Reference temperature [K] (25°C)
    β_ref = 0.004,          # Temperature coefficient [1/K]
    NOCT = 45.0,            # Nominal Operating Cell Temperature [°C]
    T_sun = 5800.0,         # Effective sun temperature [K]
)

(η_ref = 0.2, T_ref = 298.15, β_ref = 0.004, NOCT = 45.0, T_sun = 5800.0)

## Core Functions

In [2]:
"""
    SolarPV

Struct holding solar PV model parameters.
"""
struct SolarPV
    A_pv::Float64           # Panel area [m²]
    η_ref::Float64          # Reference efficiency [-]
    T_ref::Float64          # Reference temperature [K]
    β_ref::Float64          # Temperature coefficient [1/K]
    NOCT::Float64           # Nominal Operating Cell Temp [°C]
    T_sun::Float64          # Sun temperature [K]
    
    function SolarPV(A_pv::Float64;
            η_ref::Float64 = 0.20,
            T_ref::Float64 = 298.15,
            β_ref::Float64 = 0.004,
            NOCT::Float64 = 45.0,
            T_sun::Float64 = 5800.0)
        new(A_pv, η_ref, T_ref, β_ref, NOCT, T_sun)
    end
end

SolarPV

In [3]:
"""
    solar_exergy_factor(T_0::Float64, T_sun::Float64) -> Float64

Calculate Petela-Landsberg-Press solar exergy factor.
"""
function solar_exergy_factor(T_0::Float64, T_sun::Float64)
    ratio = T_0 / T_sun
    return 1.0 - (4.0/3.0) * ratio + (1.0/3.0) * ratio^4
end

solar_exergy_factor

In [4]:
"""
    cell_temperature(pv::SolarPV, T_amb::Float64, G::Float64) -> Float64

Calculate cell temperature [K] using NOCT model.
T_amb in [K], G in [W/m²]
"""
function cell_temperature(pv::SolarPV, T_amb::Float64, G::Float64)
    T_amb_C = T_amb - 273.15
    T_cell_C = T_amb_C + ((pv.NOCT - 20.0) / 800.0) * G
    return T_cell_C + 273.15  # Convert back to K
end

cell_temperature

In [5]:
"""
    pv_efficiency(pv::SolarPV, T_cell::Float64) -> Float64

Calculate temperature-dependent electrical efficiency.
"""
function pv_efficiency(pv::SolarPV, T_cell::Float64)
    η = pv.η_ref * (1.0 - pv.β_ref * (T_cell - pv.T_ref))
    return max(η, 0.0)  # Ensure non-negative
end

pv_efficiency

In [6]:
"""
    pv_power_output(pv::SolarPV, G::Float64, T_amb::Float64) -> Float64

Calculate electrical power output [W].
G: irradiance [W/m²], T_amb: ambient temperature [K]
"""
function pv_power_output(pv::SolarPV, G::Float64, T_amb::Float64)
    if G <= 0.0
        return 0.0
    end
    T_cell = cell_temperature(pv, T_amb, G)
    η = pv_efficiency(pv, T_cell)
    return η * G * pv.A_pv
end

pv_power_output

In [7]:
"""
    analyze_pv(pv::SolarPV, G::Float64, T_amb::Float64) -> NamedTuple

Complete energy and exergy analysis for given conditions.
G: irradiance [W/m²], T_amb: ambient temperature [K]
"""
function analyze_pv(pv::SolarPV, G::Float64, T_amb::Float64)
    # Handle zero irradiance
    if G <= 0.0
        return (
            G = G,
            T_amb = T_amb,
            T_cell = T_amb,
            E_solar_in = 0.0,
            W_elec = 0.0,
            Ex_solar_in = 0.0,
            Ex_elec = 0.0,
            Ex_d = 0.0,
            η_en = 0.0,
            ψ_solar = solar_exergy_factor(T_amb, pv.T_sun),
            Ψ_pv = 0.0
        )
    end
    
    # Energy analysis
    T_cell = cell_temperature(pv, T_amb, G)
    η_en = pv_efficiency(pv, T_cell)
    E_solar_in = G * pv.A_pv
    W_elec = η_en * E_solar_in
    
    # Exergy analysis
    ψ_solar = solar_exergy_factor(T_amb, pv.T_sun)
    Ex_solar_in = ψ_solar * E_solar_in
    Ex_elec = W_elec  # Electricity is pure exergy
    Ex_d = Ex_solar_in - Ex_elec
    Ψ_pv = Ex_elec / Ex_solar_in
    
    return (
        G = G,                    # Irradiance [W/m²]
        T_amb = T_amb,            # Ambient temperature [K]
        T_cell = T_cell,          # Cell temperature [K]
        E_solar_in = E_solar_in,  # Solar energy input [W]
        W_elec = W_elec,          # Electrical output [W]
        Ex_solar_in = Ex_solar_in,# Solar exergy input [W]
        Ex_elec = Ex_elec,        # Electrical exergy [W]
        Ex_d = Ex_d,              # Exergy destruction [W]
        η_en = η_en,              # Energy efficiency [-]
        ψ_solar = ψ_solar,        # Solar exergy factor [-]
        Ψ_pv = Ψ_pv               # Exergy efficiency [-]
    )
end

analyze_pv

## Tests

In [9]:
using Test

function run_pv_tests()
    println("Running Solar PV Tests...")
    
    # Create test PV system (10 m²)
    pv = SolarPV(10.0)
    
    # Test 1: Solar exergy factor
    @testset "Solar Exergy Factor" begin
        ψ = solar_exergy_factor(298.0, 5800.0)
        @test ψ ≈ 0.9327 atol=0.01  # Should be ~0.93
        @test 0.0 < ψ < 1.0
    end
    
    # Test 2: Cell temperature
    @testset "Cell Temperature" begin
        T_amb = 298.15  # 25°C
        G = 1000.0      # 1 sun
        T_cell = cell_temperature(pv, T_amb, G)
        @test T_cell > T_amb  # Cell hotter than ambient
        @test T_cell ≈ 298.15 + (45-20)/800*1000 atol=0.1
    end
    
    # Test 3: Efficiency degradation with temperature
    @testset "Temperature Effect" begin
        η_ref = pv_efficiency(pv, pv.T_ref)  # At reference
        η_hot = pv_efficiency(pv, pv.T_ref + 25.0)  # 25K hotter
        @test η_ref ≈ pv.η_ref atol=1e-6
        @test η_hot < η_ref  # Efficiency drops with temperature
    end
    
    # Test 4: Power output at STC
    @testset "Power Output" begin
        G = 1000.0
        T_amb = 298.15
        W = pv_power_output(pv, G, T_amb)
        @test W > 0.0
        @test W < G * pv.A_pv  # Less than incident power
    end
    
    # Test 5: Zero irradiance
    @testset "Zero Irradiance" begin
        result = analyze_pv(pv, 0.0, 298.15)
        @test result.W_elec == 0.0
        @test result.Ex_d == 0.0
    end
    
    # Test 6: Exergy balance
    @testset "Exergy Balance" begin
        result = analyze_pv(pv, 1000.0, 298.15)
        # Exergy balance: Ex_in = Ex_out + Ex_d
        @test result.Ex_solar_in ≈ result.Ex_elec + result.Ex_d atol=1e-6
        # For PV: Ψ = η_en / ψ_solar > η_en (since ψ_solar < 1)
        @test result.Ψ_pv > result.η_en
        @test result.Ψ_pv ≈ result.η_en / result.ψ_solar atol=1e-6
    end
    
    # Test 7: Energy efficiency bounds
    @testset "Efficiency Bounds" begin
        result = analyze_pv(pv, 1000.0, 298.15)
        @test 0.0 < result.η_en < 1.0
        @test 0.0 < result.Ψ_pv < 1.0
    end
    
    println("All tests passed!")
end

run_pv_tests()

Running Solar PV Tests...
Test Summary:       | Pass  Total  Time
Solar Exergy Factor |    2      2  0.0s
Test Summary:    | Pass  Total  Time
Cell Temperature |    2      2  0.0s
Test Summary:      | Pass  Total  Time
Temperature Effect |    2      2  0.0s
Test Summary: | Pass  Total  Time
Power Output  |    2      2  0.0s
Test Summary:   | Pass  Total  Time
Zero Irradiance |    2      2  0.0s
Test Summary:  | Pass  Total  Time
Exergy Balance |    3      3  0.0s
Test Summary:     | Pass  Total  Time
Efficiency Bounds |    2      2  0.0s
All tests passed!


## Example Usage

In [10]:
# Example: 20 m² PV array at 1000 W/m², 30°C ambient
pv = SolarPV(20.0, η_ref=0.20)
result = analyze_pv(pv, 1000.0, 303.15)

println("Solar PV Analysis")
println("="^40)
println("Panel area:         $(pv.A_pv) m²")
println("Irradiance:         $(result.G) W/m²")
println("Ambient temp:       $(round(result.T_amb - 273.15, digits=1)) °C")
println("Cell temp:          $(round(result.T_cell - 273.15, digits=1)) °C")
println("─"^40)
println("Solar energy in:    $(round(result.E_solar_in/1000, digits=2)) kW")
println("Electrical output:  $(round(result.W_elec/1000, digits=2)) kW")
println("Energy efficiency:  $(round(result.η_en*100, digits=1))%")
println("─"^40)
println("Solar exergy in:    $(round(result.Ex_solar_in/1000, digits=2)) kW")
println("Exergy destruction: $(round(result.Ex_d/1000, digits=2)) kW")
println("Exergy efficiency:  $(round(result.Ψ_pv*100, digits=1))%")

Solar PV Analysis
Panel area:         20.0 m²
Irradiance:         1000.0 W/m²
Ambient temp:       30.0 °C
Cell temp:          61.2 °C
────────────────────────────────────────
Solar energy in:    20.0 kW
Electrical output:  3.42 kW
Energy efficiency:  17.1%
────────────────────────────────────────
Solar exergy in:    18.61 kW
Exergy destruction: 15.19 kW
Exergy efficiency:  18.4%


## Export

Functions available for import:
- `SolarPV` - Model struct
- `solar_exergy_factor` - Petela-Landsberg-Press factor
- `cell_temperature` - NOCT temperature model
- `pv_efficiency` - Temperature-dependent efficiency
- `pv_power_output` - Electrical power output
- `analyze_pv` - Complete analysis